In [3]:
## 2.3 요약
# 데이터 가져오기
import os
HOUSING_PATH = os.path.join("datasets", "housing")

import pandas as pd
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()


# 중요 특성인 중간소득을 계층 별로 나눠 계층적 샘플링
import numpy as np
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

    
# 계층적 샘플링에 사용했던 income_cat 특성 제거하기
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

## 2.4는 데이터 탐색과 시각화일 뿐, 2.5에 영향을 주진 않는다

In [4]:
## 2.5 머신러닝 알고리즘을 위한 데이터 준비
'''
함수로 자동화
원래 훈련 세트로 복원하고, 예측 변수과 타깃값(레이블)을 분리한다.
drop()은 복사본을 만들며, 기존에 영향을 주지 않는다.
'''

housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [6]:
# 2.5.1 데이터 정제
'''
누락된 특성을 처리하자
1. 해당 구역 제거
2. 전체 특성 삭제
3. 어떤 값으로 채우기 -> 계산한 중간값을 저장하고 사용.
'''

#housing.dropna(subset=["total_bedrooms"])
#housing.drop("total_bedrooms", axis=1)
#median = housing["total_bedrooms"].median()
#housing["total_bedrooms"].fillna(median, inplace=True)

pass

In [7]:
'''
사이킷런의 SimpleImputer -> 누락된 값을 손쉽게 다루게 해줌
일단, 특성의 중간값으로 대체하게 해보자
'''

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [8]:
'''
중간값은 수치형 데이터에서만 계산될 수 있음 -> 텍스트 특성인 ocean_proximity를 제외한 데이터 복사본 생성
'''

housing_num = housing.drop("ocean_proximity", axis=1)

In [9]:
'''
imputer 객체의 fit() 메서드 사용해서 훈련 데이터에 적용
'''

imputer.fit(housing_num)

SimpleImputer(strategy='median')

In [10]:
'''
imputer는 각 특성의 중간값을 계산해서 그 결과를 객체의 statistics_ 속성에 저장한다
나중에 어떤 값이 누락될 지 모르니 모든 수치형 특성에 imputer를 적용하자
'''

imputer.statistics_

array([-118.51  ,   34.26  ,   29.    , 2119.5   ,  433.    , 1164.    ,
        408.    ,    3.5409])

In [11]:
housing_num.median().values

array([-118.51  ,   34.26  ,   29.    , 2119.5   ,  433.    , 1164.    ,
        408.    ,    3.5409])

In [13]:
'''
이제 학습된 imputer 객체를 사용해 훈련 세트에 누락된 값을 중간값으로 바꿀 수 있다
'''

X = imputer.transform(housing_num)
X

array([[-121.89  ,   37.29  ,   38.    , ...,  710.    ,  339.    ,
           2.7042],
       [-121.93  ,   37.05  ,   14.    , ...,  306.    ,  113.    ,
           6.4214],
       [-117.2   ,   32.77  ,   31.    , ...,  936.    ,  462.    ,
           2.8621],
       ...,
       [-116.4   ,   34.09  ,    9.    , ..., 2098.    ,  765.    ,
           3.2723],
       [-118.01  ,   33.82  ,   31.    , ..., 1356.    ,  356.    ,
           4.0625],
       [-122.45  ,   37.77  ,   52.    , ..., 1269.    ,  639.    ,
           3.575 ]])

In [15]:
'''
이 결과는 평범한 넘파이 배열이다. 다시 판다스 데이터프레임으로 되돌리자
'''

housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)